# Tarea 2

**Envío del PR inicial: 07.09.2018**

**Aceptación del PR: 14.09.2018**

## 1

Extiendan los operadores `+`, `-`, `*` y `/` para que funcionen cuando una de las dos entradas es un `Dual` y la otra es un `Real` (número cualquiera), sin importar el orden.

In [11]:
"""
    Dual

Definición de los números duales. Los campos internos son
    x  :: Float64   # valor de la función
    x′ :: Float64   # valor de su derivada

"""
struct Dual
    x  :: Float64
    x′ :: Float64
end

Dual

In [12]:
import Base: +, -, *, /, ^

### (a) Suma

In [87]:
+(u::Dual, v::Dual) = Dual( u.x + v.x, u.x′ + v.x′)

+(a::Real, v::Dual) = Dual( a + v.x, v.x′)

+(u::Dual, a::Real) = Dual( u.x + a, u.x′)

+ (generic function with 183 methods)

### (b) Resta

In [88]:
-(u::Dual, v::Dual) = Dual( u.x - v.x, u.x′ - v.x′)

-(a::Real, v::Dual) = Dual( a - v.x, - v.x′)

-(u::Dual, a::Real) = Dual( u.x - a, u.x′)

- (generic function with 197 methods)

### (c) Producto

In [89]:
*(u::Dual, v::Dual) = Dual( u.x * v.x, u.x * v.x′ + u.x′ * v.x)

*(a::Real, v::Dual) = Dual( a * v.x, a * v.x′)

*(u::Dual, v::Real) = Dual( u.x * a, u.x′ * a)

* (generic function with 185 methods)

### (d) Cociente

In [90]:
function /(u::Dual, v::Dual)
    y = u.x / v.x
    Dual( y, (u.x′ - y * v.x′)/v.x )
end

/ (generic function with 76 methods)

In [91]:
/(a::Real, v::Dual) = Dual(a / v.x, (-(a / v.x) * v.x′)/v.x)

/(u::Dual, a::Real) = Dual(u.x / a, (u.x′ / a))

/ (generic function with 76 methods)

## 2

Construyan 2 tests para cada una de las operaciones aritméticas definidas para los `Dual`es, incluyendo las que definieron en el ejercicio 1.

In [92]:
using Base.Test

### (a) Suma

In [93]:
@test Dual(1234, 5678) + Dual(4444, -4444) == Dual(5678, 1234)

Test Passed

In [94]:
@test Dual(pi, pi^4) + Dual(pi^4, -pi) == Dual(pi^4 + pi, pi^4 - pi)

Test Passed

### (b) Resta

In [95]:
@test Dual(1234, 5678) - Dual(-4444, 4444) == Dual(5678, 1234)

Test Passed

In [96]:
@test Dual(234, 789) - Dual(111, 111) == Dual(123, 678)

Test Passed

### (c) Producto

In [97]:
f(x) = 2x + 8

f (generic function with 1 method)

In [98]:
g(x) = 4x^2 + 8x + 1

g (generic function with 1 method)

In [99]:
f′(x) = 2

f′ (generic function with 1 method)

In [100]:
g′(x) = 8x + 8

g′ (generic function with 1 method)

In [101]:
@test Dual(f(1), f′(1)) * Dual(g(1), g′(1)) == Dual(f(1)*g(1), f′(1)*g(1) + f(1)*g′(1))

Test Passed

In [102]:
@test Dual(f(1), g(1)) * Dual(f(-1), g(-1)) == Dual(60, 48)

Test Passed

### (d) Cociente

In [103]:
@test Dual(f(1), f′(1)) / Dual(g(1), g′(1)) == Dual(f(1)/g(1), (f′(1)*g(1) - f(1)*g′(1))/ g(1)^2)

Test Passed

In [107]:
@test Dual(f(1.5), f′(1.5)) / 5 == Dual( f(1.5)/5, ((f′(1.5)*5)/ 5^2))

Test Passed

## 3

Sobrecarguen la función `show` (que está en Base) para que cuando se impriman los duales aparezcan con la notación $\varepsilon$ que usamos al principio de este notebook, por ejemplo, el resultado de `Dual(1,2)` debe ser parecido a `1.0 + 2.0 ε`.

In [108]:
import Base:show

In [151]:
### Definiendo funcion show ###

function show(io::IO, u::Dual)
    if u.x′ < 0
        print(u.x, " - ",abs(u.x′)," ε " )
    else 
        print(u.x," + ",u.x′," ε ")
    end
end

show (generic function with 273 methods)

In [150]:
### Probando la funcion show ###
Dual(pi^4, -pi^8 + 1)

97.40909103400243 - 9487.531016070572 ε 

## 4

Obtengan la derivada de 
$$g(x) = \frac{3x^2-8x+5}{7x^3-1}$$
en $x_0=1$.

## 5

- Recordando la regla de la cadena(!!!), extiendan el usar los `Dual` a las funciones `sqrt`, `exp`, `log`, `sin`, `cos`, `sinh` y `cosh`. 

- Muestren que las cosas dan los resultados que esperan usando pruebas como hicieron en el ejercicio 2.

- Calculen la derivada de $h(x) = \sin\Big(x^3 - \frac{2}{x^6}\Big)$ en $x_0 = 2$. ¿Qué tan preciso es el resultado?(Pueden usar cualquier otra manera de obtener el resultado correcto, sólo tienen que ser claros en la explicación.)

- Dibujen, para $x_0 \in [1,5]$ la función $h'(x)$.

## 6

Implementen el método de Newton para una función arbitraria $f(x)$ en una variable, explotando el uso de los `Dual`. 

- Obtengan, usando su función para el método de Newton, las raices del polinomio de Wilkinson 

$$
W_{6}(x) = (x-1)(x-2)(x-3)(x-4)(x-5)(x-6),
$$ 

usando como iterados iniciales $x_0=2.2$ y $x_0=2.45$.
    
- Hagan lo mismo que en el inciso anterior para $g(x) = x \sin(2x)$, con $x_0=0.7$.

## 7

Argumenten qué podrían hacer para extender la idea de los `Dual` y calcular derivadas aún más altas. Como caso concreto, piensen en querer obtener la derivada 18 de funciones como las que hemos usado arriba